In [1]:
import os
from parse_soft import ParseSoft
from search_soft import SearchSoft
from enrich_soft import EnrichSoft
from label_data import LabelData

%load_ext autoreload
%autoreload 2

In [2]:
# step 1: set defaults
geo = "GSE136831"
soft_dir = '/home/yuan/data'
meta_dir = '../data'
download_dir = '/home/yuan/rawdata/SRR'

In [3]:
# step 2: parse soft, and enrich

# analyze soft file
parser = ParseSoft(soft_dir)
soft_path = parser.soft_local_path(geo)
print('Retrieve meta data from soft file: ', soft_path)
softer = SearchSoft(soft_path)
data = softer.filter_data(softer.parse_rows)

# save to json
LabelData(meta_dir).save(data)

Retrieve meta data from soft file:  /home/yuan/data/ftp.ncbi.nlm.nih.gov/geo/series/GSE136nnn/GSE136831/soft/GSE136831_family.soft.gz
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE136nnn/GSE136831/metadata.json


In [4]:
# step 3a: enrich data
data = LabelData(meta_dir).get_meta(geo)
enriched = EnrichSoft(data)()
LabelData(meta_dir).save(enriched)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE136nnn/GSE136831/metadata.json


In [7]:
# step 3b: label samples manually
from label_sample import LabelSample

def collator(sample):
    c = LabelSample(sample)
    c._protocol('scrna-seq', '10x3v2', 'paired-end')
    if c.disease == 'copd':
        c._disease_patient('chronic obstructive pulmonary lung disease')
    elif c.disease == 'ipf':
        c._disease_patient('idiopathic pulmonary fibrosis lung disease')
    elif c.disease == 'control':
        c._healthy_control()
    c._update('tissue', 'lung')

data = LabelData(meta_dir).get_meta(geo)
for sample_id, sample in data['samples'].items():
    collator(sample)
LabelData(meta_dir).save(data)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE136nnn/GSE136831/metadata.json


In [9]:
# step 4a: load some data
from utils import Utils

# data are determined previously
samn_srr = Utils.from_json('../results/samn_srr.json')
print(len(samn_srr))

47


In [10]:
# step 4b: Given BioSample, parse SRR accessions into samples
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr(data, samn_srr)
LabelData(meta_dir).save(enriched)

biosamples = 78, SRR accessions = 107.
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE136nnn/GSE136831/metadata.json


In [11]:
# step 5: merge srr_urls into data

# sample.<sample_id>.SRR.<SRR accession>."ftp.sra.ebi.ac.uk"
file_name = 'srr_fastq_urls_simple.json'
urls_json = os.path.join('../results', file_name)
urls = Utils.from_json(urls_json)

# parse urls
data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr_urls(data, urls)
LabelData(meta_dir).save(enriched)

{'srr': 78, 'available': 107, 'unknown': 0, 'updated': 107}
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE136nnn/GSE136831/metadata.json


In [12]:
# step 6a: download raw data using biosample accession using nextflow
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_biosample(sample_iter)
# run the bash script showed here
# Outcome: nf download failed

Number of biosamples:  78
Number of biosamples with SRR:  78
bash /home/yuan/rawdata/SRR/GSE136831/fetch1.sh


In [13]:
#Step 6b: collect all SRR without local_fastq then execute bash run2.sh
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_ebi_srr(sample_iter)
# run the bash script showed here
# outcome: all ebi.ftp urls are not existing.

Number of SRR:  107
bash /home/yuan/rawdata/SRR/GSE136831/fetch2.sh


In [14]:
#Step 6c: download data using fastq-dump
# collect all SRR without local_fastq then execute bash run3.sh
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_srr(sample_iter)
# run the bash script showed here

Number of SRR:  107
bash /home/yuan/rawdata/SRR/GSE136831/fetch3.sh


In [15]:
# step 7: parse path of local fastq.gz with SRR in GEO data
# Note: run the process multiple times to make sure all SRRs are accessed
# print all samples if not any local *.fastq.gz is parsed
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
data, unparsing = ParseSra.parse_local_fastq(data, download_dir)
if unparsing:
    print(geo, len(unparsing), unparsing[:6])
LabelData(meta_dir).save(data)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE136nnn/GSE136831/metadata.json


## nf-core/scrnaseq pipeline
create samplehseet.csv and params.config
3 samples

In [32]:
# scrnaseq_lung_cell_line
from utils import Utils
from label_data import LabelData
from label_sample import LabelSample

name = 'scrnaseq_lung_tissue_normal'
output_dir = '/home/yuan/output'
labeler = LabelData(meta_dir, name, output_dir, True)

sample_sheet = {}
data = LabelData(meta_dir).get_meta(geo)
for sample, geo, run_acc, fastq_sample in labeler.fastq_iter(data):
    cl = LabelSample(sample)
    if cl.filter(name):
        keys = [cl.cell_line(), geo, run_acc,]
        Utils.key_update(sample_sheet, [geo,], fastq_sample['sample_sheet'])
# export
if sample_sheet:
    print("Number of samples in samplesheet.csv: ", len(sample_sheet[geo]))
    labeler.to_sample_sheet(sample_sheet, 'paired')
    labeler.nf_cmd()
else:
    print(f'No samples meet creteria {name}')

No samples meet creteria scrnaseq_lung_tissue_normal


In [38]:
name = 'scrnaseq_lung_tissue_patient'
output_dir = '/home/yuan/output'
labeler = LabelData(meta_dir, name, output_dir, True)

sample_sheet = {}
data = LabelData(meta_dir).get_meta(geo)
for sample, geo, run_acc, fastq_sample in labeler.fastq_iter(data):
    cl = LabelSample(sample)
    print(sample['sample_id'], cl.filter(name))
    if cl.filter(name):
        keys = [cl.cell_line(), geo, run_acc,]
        Utils.key_update(sample_sheet, [geo,], fastq_sample['sample_sheet'])
# export
if sample_sheet:
    print("Number of samples in samplesheet.csv: ", len(sample_sheet[geo]))
    labeler.to_sample_sheet(sample_sheet, 'paired')
    labeler.nf_cmd()
else:
    print(f'No samples meet creteria {name}')

GSE135851
No samples meet creteria scrnaseq_lung_tissue_patient
